<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/stock_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import torch
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Microsoft_Stock.csv')

In [ ]:
# 날짜데이터를 누락.
data.drop('Date', axis=1, inplace=True)

In [ ]:
# 데이터 프레임을 넘파이배열로 변환
n_data = data.to_numpy()
n_data.shape

In [ ]:
# data 생성기. 랜던한날짜로부터 세달(60일)치 data를 인덱싱하고 다음한달간의 종가변화율을 라벨로하는 데이터 생성.
#              Size는 data 갯수.
def data_gen(Size:int):

    s = np.random.choice(range(0, 1511-81), size=Size, replace=False)
    s = np.array(s)

    short_term_data = []
    short_term_label = []

    for i in s:
        s_data = n_data[i:i+60, :]
        short_term_data.append(s_data)

        label = (n_data[i+81,2] - n_data[i+61,2]) / n_data[i+61,2] * 100

        if label >= 10:
            short_term_label.append(3)

        elif 0 <= label < 10:
            short_term_label.append(2)

        elif -10 < label < 0:
            short_term_label.append(1)

        elif label <= -10:
            short_term_label.append(0)

    short_term_data = np.array(short_term_data).astype(np.float32)
    short_term_label = np.array(short_term_label)

    return short_term_data, short_term_label

short_term_data, short_term_label = data_gen(1200)

print(short_term_data.shape)
print(short_term_label.shape)

In [ ]:
# 데이터 표준화. 주가데이터는 지수적으로 증가하므로 각 샘플마다 따로 표준화함.
for sample in range(short_term_data.shape[0]):
    scaler = StandardScaler()
    std = scaler.fit_transform(short_term_data[sample,:,:])
    short_term_data[sample,:,:] = std

In [ ]:
short_term_data[0].dtype

In [ ]:
# # 2차원 데이터 표준화 예시. 기본적으로 열을 기준으로 표준화한다.
# test_data = np.array([[1, 4, 2], [4, 10, 6], [7, 16, 9]])

# # StandardScaler 초기화 및 적용
# scaler = StandardScaler()
# test_std_data = scaler.fit_transform(test_data)

# test_std_data

In [ ]:
# 데이터셋 구성.
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 인스턴스 생성.
custom_dataset = CustomDataset(short_term_data, short_term_label)

#데이터를 훈련 데이터와 검증 데이터, 테스트 데이터로 나누기. 파이토치데이터셋 인스턴스에 사용하면 라벨도 같이 분류됨.
#훈련세트 80%, 테스트세트 20% 분류
train_size = int(0.8 * len(short_term_data))
test_size = len(short_term_data) - train_size

train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

#테스트세트를 테스트세트 10%, 검증세트 10% 분류
test_size = int(0.5 * len(test_dataset))
val_size = len(test_dataset) - test_size

test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

# 데이터 로더 생성.
batch_size = 4 #데이터셋이 작은데 배치사이즈가 너무크면 문제가생김. 훈련 loss,와 검증 loss 역전, loss 하락폭 감소등 확인.
               #배치사이즈가 크면 gpu에 많은 data를 올릴 수 있어 훈련이 빨라지짐.
train_DL = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_DL = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_DL = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 데이터 로더를 통해 데이터 확인
for features, labels in train_DL:
    print(features.shape)
    print(labels)
    break

In [ ]:
print(len(train_DL.dataset))
print(len(test_DL.dataset))
print(len(val_DL.dataset))

In [ ]:
#단기모델(연속2일의 data를 받아 다음날의 주가상승률 예측)의 label생성
#주가가 지수적으로 증가하므로 증가율로 설정

#--------------------회귀모델----------------------
#!!!파라미터가 너무 많은 경우 over fitting이 빠르게 일어남.
#!!!파라미터를 410까지 줄였더니 validation의 로스가 train loss를 처음부터 역전함. 데이터 누수, 과소적함, 데이터 복잡도 부족등 검토.
#!!!파라미터 개수는 620개에서 가장 바람직한 학습패턴을 보임.
#!!!같은 조건에서 학습해도 완전히 다른 학습양상을 보이기도함.
# 회귀모델의 경우 훈련시 오차함수로 mse를 사용하고 측정시 mae를 사용하는게 유리. mse는 오차값을 크게 가져가기때문에 역전파시 유리.
#!!!회귀모델은 학습이 거의 이루어지지 않는 것 같음. 분류모델로 수정 검토.

#--------------------이진분류모델--------------------
#역시 파라미터수가 너무 많으면 안됨.
#파라미터수를 줄이니 에폭을 많이 늘릴 수 있었음.
#커널을 이진분류모델로 훈련해도 단기특성을 학습할 수 있을까? 만약 된다면 중기, 단기모델에 이진분류모델로 훈련한 커널을 적용하고 모델은 회귀모델로 작성 가능할까?
#파라미터수를 유지하면서 층을 늘려 비선형성을 키우면 overfitting됨...
#------------------------------------------------------
#너무 단기적인 주가 변동은 연관이 없을수도? rnn, lstm을 사용해보자. 단일 커널의 학습을 포기하고, 3개월이나 6개월단위의 모델을 만들거나, 로스를 여러번 받아 학습하는 방법을 생각해봐야할듯.

# class term2_model(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.linear = nn.Sequential(nn.Linear(10,15), nn.ReLU(),
#                                     nn.Linear(15,10), nn.ReLU(),
#                                     nn.Linear(10,1), nn.Sigmoid())

#     def forward(self, x):

#         x = self.linear(x)

#         return x


class ShortTermModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, 20, 5, stride=2),
                                   nn.BatchNorm2d(20),
                                   nn.ReLU())

        self.conv2 = nn.Sequential(nn.Conv2d(1, 40, kernel_size=(20, 5), stride=2),
                                   nn.BatchNorm2d(40),
                                   nn.ReLU())

        self.conv3 = nn.Sequential(nn.Conv2d(1, 80, kernel_size=(40, 5), stride=2),
                                   nn.BatchNorm2d(80),
                                   nn.ReLU())

        self.conv4 = nn.Sequential(nn.Conv2d(1, 160, kernel_size=(80, 5), stride=1),
                                   nn.BatchNorm2d(160),
                                   nn.ReLU())

        self.conv5 = nn.Sequential(nn.Conv2d(1, 320, kernel_size=(160, 5), stride=1),
                                   nn.BatchNorm2d(320),
                                   nn.ReLU())

        self.conv6 = nn.Sequential(nn.Conv2d(1, 640, kernel_size=(320, 5), stride=1),
                                   nn.BatchNorm2d(640),
                                   nn.ReLU())

        self.conv7 = nn.Sequential(nn.Conv2d(1, 1280, kernel_size=(640, 5), stride=1),
                                   nn.BatchNorm2d(1280),
                                   nn.ReLU())

        self.linear = nn.Sequential(nn.Linear(320,100), nn.ReLU(),
                                    nn.Linear(100,50), nn.ReLU(),
                                    nn.Linear(50,4))

    def forward(self, x):

        x = x.permute(0,2,1)
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = x.squeeze().unsqueeze(1)
        x = self.conv2(x)
        x = x.squeeze().unsqueeze(1)
        x = self.conv3(x)
        x = x.squeeze().unsqueeze(1)
        # x = self.conv4(x)
        # x = x.squeeze().unsqueeze(1)
        # x = self.conv5(x)
        # x = x.squeeze().unsqueeze(1)
        # x = self.conv6(x)
        # x = x.squeeze().unsqueeze(1)
        # x = self.conv7(x)
        x = torch.flatten(x, start_dim=1)
        # x = self.linear(x)



        return x

In [ ]:
for i in train_DL:
    test_data = i[0]
    break

test_data.shape

In [ ]:
model = ShortTermModel()
a = model(test_data)
print(a.shape)
print(a)

In [ ]:
i[1]

In [ ]:
model = ShortTermModel().to(DEVICE)
criterion = nn.CrossEntropyLoss()
LR = 1e-6 # -1 인경우 한번에 0또는 1만 출력하는 로컬 미니멈으로 수렴.
optimizer = optim.Adam(model.parameters(), lr = LR)
EPOCH = 200

In [ ]:
def loss_epoch(model, DL, criterion, optimizer = None):

    N = len(DL.dataset) # the number of data
    rloss = 0; rcorrect = 0

    for x_batch, y_batch in DL:

        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        # inference
        y_hat = model(x_batch)

        # loss
        loss = criterion(y_hat,y_batch)

        # update
        if optimizer is not None:
            optimizer.zero_grad() # gradient 누적을 막기 위한 초기화
            loss.backward() # backpropagation
            optimizer.step() # weight update

        # loss accumulation
        loss_b = loss.item() * x_batch.shape[0] # batch loss # BATCH_SIZE를 곱하면 마지막 18개도 32개를 곱하니까..
        rloss += loss_b # running loss
        # accuracy accumulation
        pred = y_hat.argmax(dim=1)
        corrects_b = torch.sum(pred == y_batch).item()
        rcorrect += corrects_b

    loss_e = rloss/ N # epoch loss
    accuracy_e = rcorrect/N*100

    return loss_e, accuracy_e, rcorrect

In [ ]:
# loss_epoch(model, val_DL, criterion, optimizer = optimizer)

In [ ]:
def Train(model, train_DL, val_DL, criterion, optimizer, EPOCH):

    loss_history = {"train":[], "val":[]}
    acc_history = {"train":[], "val":[]}

    for ep in tqdm(range(EPOCH), leave=False):

        model.train() # train mode로 전환
        train_loss, train_acc, _ = loss_epoch(model, train_DL, criterion, optimizer)
        loss_history["train"] += [train_loss]
        acc_history["train"] += [train_acc]

        model.eval() # test mode로 전환
        with torch.no_grad():
            val_loss, val_acc, _ = loss_epoch(model, val_DL, criterion)
            loss_history["val"] += [val_loss]
            acc_history["val"] += [val_acc]

        # # print loss
        # print(f"train loss: {round(train_loss,5)}, "
        #       f"val loss: {round(val_loss,5)} \n"
        #       f"train acc: {round(train_acc,1)} %, "
        #       f"val acc: {round(val_acc,1)} %, time: {round(time.time()-epoch_start)} s")
        # print("-"*20)

    return loss_history, acc_history

In [ ]:
loss, acc = Train(model, train_DL, val_DL, criterion, optimizer, EPOCH)

In [ ]:
plt.plot(range(1, EPOCH + 1), loss['train'], label='Train Loss')
plt.plot(range(1, EPOCH + 1), loss['val'], label='Validation Loss')

plt.xlabel('Epoch')
plt.ylabel('loss')

plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(range(1, EPOCH + 1), acc['train'], label='Train acc')
plt.plot(range(1, EPOCH + 1), acc['val'], label='Validation acc')

plt.xlabel('Epoch')
plt.ylabel('acc')

plt.title('Training and Validation acc')
plt.legend()

plt.show()

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
apple = yf.Ticker("AAPL")
hist = apple.history(period="10y")

In [ ]:
hist = hist.reset_index()
hist

In [ ]:
# 지금있는 데이터셋으로는 test mae가 0.3밑으로 안내려감.
model = model.to(DEVICE)
Test(model, test_DL)

In [ ]:
# 개별확인
model = model.to('cpu')
for i in range(10):
    print(model(test_DL.dataset[i][0]))
    print(test_DL.dataset[i][1])

In [ ]:
# 훈련시킨 필터 추출.

C_W1 = model.linear[0].weight
C_B1 = model.linear[0].bias

print(C_W1.shape)
print(C_B1)

In [ ]:
train_DL.dataset[:][0].shape

In [ ]:
model.linear[0](train_DL.dataset[0][0])

In [ ]:
n = model.linear[0](train_DL.dataset[:][0])
n = n.detach().numpy()

In [ ]:
# term3_data
# 연속된 2개의 data를 묶어서 다시 구성.
term3_data = []
term3_label = []

label = data['Close'].to_numpy() # label은 마지막날종가와 다음날종가의 증가율로.

for i in range(len(n)-1):

    d = np.hstack([n[i,:], n[i+1,:]])
    l = (label[i+3]-label[i+2])/label[i+2]*100

    term3_data.append(d)
    term3_label.append(l)

term3_data = torch.tensor(term3_data).float()    # double형은 모델훈련시 가중치와 data형이 맞지 않으므로
term3_label = torch.tensor(term3_label).float()  # float형으로 변환한다.

print(term3_data.shape)
print(term3_label.shape)
print(term3_data[0])
print(term3_label[0])

In [ ]:
self.conv_block1 = nn.Sequential(nn.Conv2d(3,32,3,padding=1),
                                         nn.BatchNorm2d(32),
                                         nn.ReLU(),
                                         nn.Conv2d(32,32,3,padding=1),
                                         nn.BatchNorm2d(32),
                                         nn.ReLU())
        self.Maxpool1 = nn.MaxPool2d(2)